In [38]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
import seaborn as sns

# Configure GPU memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define paths
BASE_DIR = '.'
MODEL_PATH = os.path.join(BASE_DIR, 'ferplus_model.h5')
DATASET_PATH = os.path.join(BASE_DIR, 'archive', 'DATASET')
TEST_LABELS_PATH = os.path.join(BASE_DIR, 'archive', 'test_labels.csv')
FINETUNED_MODEL_PATH = os.path.join(BASE_DIR, 'Fine-tuned model', 'fairness_aware_model.h5')

# Constants
IMG_SIZE = 48
BATCH_SIZE = 32
NUM_CLASSES = 7
NUM_GENDERS = 2
NUM_AGE_GROUPS = 6

# Create output directory if it doesn't exist
os.makedirs(os.path.join(BASE_DIR, 'Fine-tuned model'), exist_ok=True)

print("TensorFlow version:", tf.__version__)

# Mappings
EMOTION_MAP = {
    1: 'Surprise',
    2: 'Fear',
    3: 'Disgust',
    4: 'Happy',
    5: 'Sad',
    6: 'Angry',
    7: 'Neutral'
}

AGE_GROUP_MAP = {
    0: '0-3',
    1: '4-19',
    2: '20-39',
    3: '40-69',
    4: '70+',
    5: 'Unknown'
}

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE))
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Normalize to [0,1]
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def prepare_dataset():
    # Load labels
    df = pd.read_csv(TEST_LABELS_PATH)
    print("Total samples in dataset:", len(df))
    
    # Initialize lists to store data
    images = []
    emotion_labels = []
    gender_labels = []
    age_labels = []
    
    # Process each image
    for idx, row in df.iterrows():
        if idx % 100 == 0:
            print(f"Processing image {idx}/{len(df)}")
            
        # Construct image path
        img_path = os.path.join(DATASET_PATH, 'test', str(row['label']), row['image'])
        
        # Load and preprocess image
        img_array = load_and_preprocess_image(img_path)
        if img_array is not None:
            images.append(img_array)
            
            # Convert labels to one-hot encoding
            emotion_label = tf.keras.utils.to_categorical(row['label']-1, NUM_CLASSES)
            gender_label = tf.keras.utils.to_categorical(row['Gender'], NUM_GENDERS)
            age_label = tf.keras.utils.to_categorical(row['Age_Group'], NUM_AGE_GROUPS)
            
            emotion_labels.append(emotion_label)
            gender_labels.append(gender_label)
            age_labels.append(age_label)
    
    # Convert lists to numpy arrays
    X = np.array(images)
    y_emotion = np.array(emotion_labels)
    y_gender = np.array(gender_labels)
    y_age = np.array(age_labels)
    
    print("\nDataset shapes:")
    print(f"Images: {X.shape}")
    print(f"Emotion labels: {y_emotion.shape}")
    print(f"Gender labels: {y_gender.shape}")
    print(f"Age labels: {y_age.shape}")
    
    # Print label distribution
    print("\nLabel distributions:")
    print("Emotions:", np.sum(y_emotion, axis=0))
    print("Gender:", np.sum(y_gender, axis=0))
    print("Age groups:", np.sum(y_age, axis=0))
    
    return X, y_emotion, y_gender, y_age

# Load and prepare the dataset
print("Loading and preparing dataset...")
X, y_emotion, y_gender, y_age = prepare_dataset()

# Split the data into training and validation sets
split_idx = int(0.8 * len(X))
X_train, X_val = X[:split_idx], X[split_idx:]
y_emotion_train, y_emotion_val = y_emotion[:split_idx], y_emotion[split_idx:]
y_gender_train, y_gender_val = y_gender[:split_idx], y_gender[split_idx:]
y_age_train, y_age_val = y_age[:split_idx], y_age[split_idx:]

print("\nTraining set shapes:")
print(f"X_train: {X_train.shape}")
print(f"y_emotion_train: {y_emotion_train.shape}")
print(f"y_gender_train: {y_gender_train.shape}")
print(f"y_age_train: {y_age_train.shape}")

print("\nValidation set shapes:")
print(f"X_val: {X_val.shape}")
print(f"y_emotion_val: {y_emotion_val.shape}")
print(f"y_gender_val: {y_gender_val.shape}")
print(f"y_age_val: {y_age_val.shape}")


TensorFlow version: 2.15.1
Loading and preparing dataset...
Total samples in dataset: 3068
Processing image 0/3068
Processing image 100/3068
Processing image 200/3068
Processing image 300/3068
Processing image 400/3068
Processing image 500/3068
Processing image 600/3068
Processing image 700/3068
Processing image 800/3068
Processing image 900/3068
Processing image 1000/3068
Processing image 1100/3068
Processing image 1200/3068
Processing image 1300/3068
Processing image 1400/3068
Processing image 1500/3068
Processing image 1600/3068
Processing image 1700/3068
Processing image 1800/3068
Processing image 1900/3068
Processing image 2000/3068
Processing image 2100/3068
Processing image 2200/3068
Processing image 2300/3068
Processing image 2400/3068
Processing image 2500/3068
Processing image 2600/3068
Processing image 2700/3068
Processing image 2800/3068
Processing image 2900/3068
Processing image 3000/3068

Dataset shapes:
Images: (3068, 48, 48, 1)
Emotion labels: (3068, 7)
Gender labels: 

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss)

# Train model
model.fit(train_dataset, epochs=10)

NameError: name 'train_dataset' is not defined

In [ ]:
# Evaluate on validation/test set
test_loss, test_emotion_acc, test_demo_acc = model.evaluate(test_dataset)

print(f"Emotion Accuracy: {test_emotion_acc:.4f}")
print(f"Demographic Fairness Accuracy: {test_demo_acc:.4f}")